In [ ]:
import model
from Augment import train_datagen
import joblib
import shelve
import tensorflow as tf
import time
from math import ceil
from config import vocabulary,batch_size,n_epochs,resume_epoch,save_epoch
import helper
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
mount_point = '../'

In [ ]:
with shelve.open(mount_point+'IAM_Data','c') as shelf:
    train_label = shelf['train_label']

In [ ]:
train_array = joblib.load(mount_point+'data/train_array')

In [ ]:
graph,dropout_conv,dropout_lstm,dropout_fc,inputs,time_steps,targets,loss,train,decoded,label_error_rate,seq_len = model.model()

In [ ]:
!rm -rf ../Augmented/*

In [ ]:
train_generator = train_datagen.flow(train_array,train_label,batch_size,save_to_dir=mount_point+'Augmented',save_prefix='transformed')

In [ ]:
num_training_samples = train_array.shape[0]

In [ ]:
num_batches = int(ceil(num_training_samples/batch_size))

In [ ]:
with tf.Session(graph = graph) as sess:
        
    saver = tf.train.Saver(max_to_keep=None)
    
    sess.run(tf.global_variables_initializer())
    checkpoint = False
    timer  = 0

    #Resume training from resume_epoch
    if resume_epoch != 0:
        saver.restore(sess, mount_point+'saved_models/cnn_lstm_fc_'+str(resume_epoch))
    
    #Epoch Loop
    for e in range(resume_epoch,n_epochs):
        start_time = time.time()
        
        if (e%1)==0:
            checkpoint = True
        
        total_cost = 0.0
        total_ler = 0.0    
    
        count = 0
        
        #Mini Batch loop
        for x,y in train_generator:
            
            actual_batch_size = x.shape[0]
            
            if count == num_batches:
                count = 0
                break

            sparse_y = helper._batch_y(y,vocabulary)

            feed_train = {
                         inputs:x,targets:sparse_y,
                         time_steps:np.array([seq_len]*actual_batch_size),
                         dropout_conv:0.5,dropout_fc:0.5,dropout_lstm:0.5,
                    }
            
            _,cost_val,train_ler_val = sess.run([train,loss,label_error_rate],feed_dict= feed_train)

            print(cost_val,train_ler_val)
            
            total_cost += cost_val
            total_ler += train_ler_val

            count+=1
            
        if (e%save_epoch)==0:
            
            #Save the model
            saver.save(sess,mount_point+'saved_models/cnn_lstm_fc_'+str(e))
        
        
#         if checkpoint:
            
#             end_time = time.time()
#             time_taken = end_time - start_time
#             timer += time_taken

#             print(total_cost,total_ler)        

In [ ]:
%matplotlib inline

In [ ]:
# img = train_array[0].reshape(train_array[0].shape[:2])

In [ ]:
# plt.imshow(img,cmap='gray')

In [ ]:
# train_label[0]